In [2]:
#from bs4 import BeautifulSoup
import csv
import numpy as np
import pandas as pd
#import re
import time
from tqdm import tqdm
import json
from datetime import datetime, timedelta, timezone

In [37]:
df = pd.read_csv("../data/jinshin_archive_sentence.csv",index_col=0)
df["day"] = df.date.apply(lambda x: datetime.strptime(x, '%Y/%m/%d %H:%M').strftime("%A"))
df.date = df.date.apply(lambda x: datetime.strptime(x, '%Y/%m/%d %H:%M'))
df.date = df.date.apply(lambda x: x.strftime("%H:%M"))
df.sort_values("date", ascending=True, inplace=True)

a = df.to_numpy()
mon = {}
tue = {}
wed = {}
thu = {}
fri = {}
sat = {}
sun = {}

for i in a:
    if i[7] == "Monday":
        if i[0] in mon:
            mon[i[0]] += 1
        else:
            mon[i[0]] = 1
    elif i[7] == "Tuesday":
        if i[0] in tue:
            tue[i[0]] += 1
        else:
            tue[i[0]] = 1
    elif i[7] == "Wednesday":
        if i[0] in wed:
            wed[i[0]] += 1
        else:
            wed[i[0]] = 1
    elif i[7] == "Thursday":
        if i[0] in thu:
            thu[i[0]] += 1
        else:
            thu[i[0]] = 1
    elif i[7] == "Friday":
        if i[0] in fri:
            fri[i[0]] += 1
        else:
            fri[i[0]] = 1
    elif i[7] == "Saturday":
        if i[0] in sat:
            sat[i[0]] += 1
        else:
            sat[i[0]] = 1
    else:
        if i[0] in sun:
            sun[i[0]] += 1
        else:
            sun[i[0]] = 1

with open("timeonly_mon.json", "w") as f:
    jsonhoge = json.dump(mon, f, indent=2)
with open("timeonly_tue.json", "w") as f:
    jsonhoge = json.dump(tue, f, indent=2)
with open("timeonly_wed.json", "w") as f:
    jsonhoge = json.dump(wed, f, indent=2)
with open("timeonly_thu.json", "w") as f:
    jsonhoge = json.dump(thu, f, indent=2)
with open("timeonly_fri.json", "w") as f:
    jsonhoge = json.dump(fri, f, indent=2)
with open("timeonly_sat.json", "w") as f:
    jsonhoge = json.dump(sat, f, indent=2)
with open("timeonly_sun.json", "w") as f:
    jsonhoge = json.dump(sun, f, indent=2)


In [44]:
def inputation(filename):
    with open(filename,"r") as f:
        a = json.load(f)
        for hour in range(24):
            for minute in range(60):
                key = str(hour).zfill(2) + ":" + str(minute).zfill(2)
                if  not key in a:
                    a[key] = 0
    with open(filename,"w") as f:
        json.dump(a,f, indent=2)

In [45]:
inputation("timeonly_mon.json")
inputation("timeonly_tue.json")
inputation("timeonly_wed.json")
inputation("timeonly_thu.json")
inputation("timeonly_fri.json")
inputation("timeonly_sat.json")
inputation("timeonly_sun.json")

In [47]:
def clockdata(filename1, filename2):
    section = [0, 10, 20, 30, 40, 50, 60]
    res = []
    with open(filename1, "r") as f:
        d = json.load(f)
        for hour in range(24):
            for i in range(len(section)-1):
                start = section[i]
                end = section[i+1]
                dataarray = []
                for minute in range(start, end):
                    key = str(hour).zfill(2) + ":" + str(minute).zfill(2)
                    dataarray.append(d[key])
                tmp = np.array(dataarray)
                c_array = np.percentile(tmp,[0,25,50,75,100])
                res.append({"hour": hour, "minute": section[i], "min": c_array[0], "quarter":c_array[1], "median": c_array[2], "quarter3": c_array[3], "max": c_array[4]})
    with open(filename2,"w") as f:
        json.dump(res,f,indent=2)

In [49]:
clockdata("timeonly_mon.json", "clockdata_mon.json")
clockdata("timeonly_tue.json", "clockdata_tue.json")
clockdata("timeonly_wed.json", "clockdata_wed.json")
clockdata("timeonly_thu.json", "clockdata_thu.json")
clockdata("timeonly_fri.json", "clockdata_fri.json")
clockdata("timeonly_sat.json", "clockdata_sat.json")
clockdata("timeonly_sun.json", "clockdata_sun.json")

In [41]:
door = []
with open("doorR2_converted.csv") as f:
  reader = csv.reader(f)
  for row in reader:
    door.append(row)
del door[0]

doorday = {}
for i in door:
  i[8] = i[8][:10]
  if i[8] in doorday:
    doorday[i[8]] += 1
  else:
    doorday[i[8]] = 1
doorday = sorted(doorday.items())
del doorday[-1]

for i in range(len(doorday)):
  doorday[i] = list(doorday[i])
  if i > 0:
    doorday[i][1] = doorday[i-1][1] + doorday[i][1]

#print(doorday)

with open("doordate.csv", "w") as f:
  writer = csv.writer(f)
  writer.writerow(["date", "value"])
  for k, v in doorday:
    writer.writerow([k, v])

[['1974/01/01', 1], ['1977/12/01', 2], ['1981/02/01', 11], ['1981/03/01', 19], ['1985/01/01', 20], ['1985/03/01', 22], ['1989/07/01', 36], ['1990/02/01', 42], ['1990/03/01', 43], ['1991/11/01', 50], ['1994/08/01', 71], ['1995/11/01', 82], ['1996/03/01', 88], ['1997/09/01', 90], ['1997/10/01', 106], ['1997/12/01', 108], ['1998/08/01', 111], ['1998/11/01', 119], ['2000/01/01', 130], ['2000/04/01', 133], ['2000/05/01', 138], ['2000/06/01', 144], ['2000/07/01', 149], ['2000/08/01', 164], ['2000/09/01', 170], ['2001/03/01', 177], ['2001/06/01', 178], ['2001/07/01', 183], ['2002/02/01', 184], ['2002/09/01', 194], ['2002/11/01', 195], ['2002/12/01', 198], ['2003/03/01', 201], ['2003/08/01', 216], ['2003/10/01', 217], ['2004/03/01', 235], ['2004/05/01', 239], ['2004/10/01', 250], ['2004/11/01', 252], ['2004/12/01', 253], ['2005/02/01', 269], ['2005/03/01', 278], ['2005/08/01', 298], ['2006/01/01', 305], ['2006/02/01', 308], ['2006/03/01', 312], ['2006/04/01', 313], ['2006/06/01', 315], ['2006/